# TESTE FUNÇÕES

## Bibliotecas

In [107]:
import pandas as pd
from openpyxl import load_workbook
import os
from datetime import datetime, timedelta
import pdfplumber
import re

## Funções

In [148]:
gestao_monitoramento = 'gestao_monitoramento.xlsx'
# salva formatação
workbook = load_workbook(gestao_monitoramento)
df_clientes = pd.read_excel(gestao_monitoramento, sheet_name='config_clientes', index_col=None)
df_cert_config = pd.read_excel(gestao_monitoramento, sheet_name='config_certidoes', index_col=None)
df_monitor = pd.read_excel(gestao_monitoramento, sheet_name='db_monitor')

### certidao_cliente(df_clientes, df_cert_config)
cria um dataframe clientes 1:1 certidões

In [149]:
def certidoes_clientes_ativo(df_clientes, df_cert_config):
    try:
        df_clientes_expandido = df_clientes.assign(
            CERTIDAO_ID=df_clientes['CERTIDOES_CADASTRADAS'].str.split(', ')
        ).explode('CERTIDAO_ID')

        df_clientes_certidoes = pd.merge(df_clientes_expandido,
                                    df_cert_config,
                                    on='CERTIDAO_ID', 
                                    how='left')[['CERTIDAO_ID', 'CLIENTE_ID', 'ATIVO', 'PASTA_DESTINO', 'NOME_ARQUIVO']]

        df_clientes_certidoes = df_clientes_certidoes.rename(columns={'NOME_ARQUIVO': 'ARQUIVO'})
        
        # filtrando apenas onde o 'ATIVO' é True
        df_clientes_certidoes = df_clientes_certidoes.loc[df_clientes_certidoes['ATIVO'] == True]
        
        print('df_cliente_certidao criado com sucesso')
        return df_clientes_certidoes
    except Exception as e:
        print(f'erro ao gerar df_clientes_certidoes {e}')
        return None
    
df_cliente_certidao = certidoes_clientes_ativo(df_clientes, df_cert_config)

df_cliente_certidao criado com sucesso


In [154]:
df_cliente_certidao.info()
df_cliente_certidao.head()

<class 'pandas.core.frame.DataFrame'>
Index: 45 entries, 0 to 44
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   CERTIDAO_ID    45 non-null     object
 1   CLIENTE_ID     45 non-null     object
 2   ATIVO          45 non-null     bool  
 3   PASTA_DESTINO  45 non-null     object
 4   ARQUIVO        45 non-null     object
dtypes: bool(1), object(4)
memory usage: 1.8+ KB


,CERTIDAO_ID,CLIENTE_ID,ATIVO,PASTA_DESTINO,ARQUIVO
0,CEF-CRFGTS,AUTOLTDA,True,clientes/autodata/,CR FGTS
1,TJDFT-CNFR,AUTOLTDA,True,clientes/autodata/,CN Falencia e Recuperacao
2,CGU-CNC,AUTOLTDA,True,clientes/autodata/,"CN Correcional (ePAD, CGU-PJ, CEIS, CNEP e CEPIM)"
3,FN-CNDTF,AUTOLTDA,True,clientes/autodata/,CND Tributos Federais
4,GDF-CND,AUTOLTDA,True,clientes/autodata/,CND GDF


### verificar_existencia_arquivos(dataframe)
recebe um DF (df_cliente_certidao ou df_monitor) e retorna df com coluna bool 'EXISTE'

In [ ]:
def verificar_existencia_arquivos(dataframe):
    try:
        resultados = []
        pastas_grupo = dataframe.groupby('PASTA_DESTINO')

        for pasta_destino, grupo in pastas_grupo:
            # verificar se o diretório existe
            if not os.path.isdir(pasta_destino):
                for _, row in grupo.iterrows():
                    resultados.append({
                        'CERTIDAO_ID': row['CERTIDAO_ID'],
                        'CLIENTE_ID': row['CLIENTE_ID'],
                        'EXISTE': False,
                        'ARQUIVO': row['ARQUIVO'],
                        'PASTA_DESTINO': pasta_destino
                    })
                continue

            # lista todos os arquivos da pasta_destino
            arquivos_na_pasta = os.listdir(pasta_destino)

            for _, row in grupo.iterrows():
                nome_arquivo = row['ARQUIVO']
                arquivos_encontrados = []

                for arquivo in arquivos_na_pasta:
                    if nome_arquivo in arquivo and arquivo.endswith('.pdf'):
                        arquivo_pdf = os.path.join(arquivo)
                        arquivos_encontrados.append(arquivo_pdf)

                # Adicionar uma entrada para cada arquivo encontrado
                if arquivos_encontrados:
                    for arquivo_pdf in arquivos_encontrados:
                        resultados.append({
                            'CERTIDAO_ID': row['CERTIDAO_ID'],
                            'CLIENTE_ID': row['CLIENTE_ID'],
                            'EXISTE': True,
                            'ARQUIVO': arquivo_pdf,
                            'PASTA_DESTINO': pasta_destino
                        })
                else:
                    # Se nenhum arquivo foi encontrado, adicionar como não encontrado
                    resultados.append({
                        'CERTIDAO_ID': row['CERTIDAO_ID'],
                        'CLIENTE_ID': row['CLIENTE_ID'],
                        'EXISTE': False,
                        'ARQUIVO': row['ARQUIVO'],
                        'PASTA_DESTINO': pasta_destino
                    })
        #cria df com os resultados encontrados
        df_clientes_certidoes_resultado = pd.DataFrame(resultados)
        return df_clientes_certidoes_resultado
    except Exception as e:
        print(f'Erro ao verificar a existência dos arquivos {e}')

df_cliente_certidao_resultado = verificar_existencia_arquivos(df_cliente_certidao)

In [153]:
df_cliente_certidao_resultado.info()
df_cliente_certidao_resultado.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   CERTIDAO_ID    45 non-null     object
 1   CLIENTE_ID     45 non-null     object
 2   EXISTE         45 non-null     bool  
 3   ARQUIVO        45 non-null     object
 4   PASTA_DESTINO  45 non-null     object
dtypes: bool(1), object(4)
memory usage: 1.6+ KB


,CERTIDAO_ID,CLIENTE_ID,EXISTE,ARQUIVO,PASTA_DESTINO
0,CEF-CRFGTS,AUTOLTDA,True,2024-09-30_CR FGTS.pdf,clientes/autodata/
1,TJDFT-CNFR,AUTOLTDA,True,2024-09-09_CN Falencia e Recuperacao.pdf,clientes/autodata/
2,CGU-CNC,AUTOLTDA,True,"2024-09-13_CN Correcional (ePAD, CGU-PJ, CEIS,...",clientes/autodata/
3,FN-CNDTF,AUTOLTDA,True,2024-09-06_CND Tributos Federais.pdf,clientes/autodata/
4,GDF-CND,AUTOLTDA,True,2024-09-20_CND GDF.pdf,clientes/autodata/


In [132]:
log_erros =[]

### arquivos_novos_dbmonitor(df_monitor, dataframe)

In [175]:
def arquivos_novos_dbmonitor(df_monitor, dataframe):
    horario_atual = datetime.now().replace(microsecond=0)#.strftime('%Y-%m-%d %H:%M:%S')

    certidao_nao_encontrada = []
    certidao_adicionada = []
    

    # Itera sobre cada linha do DataFrame df_resultados
    for _, row in dataframe.iterrows():
        certidao_id = row['CERTIDAO_ID']
        cliente_id = row['CLIENTE_ID']
        
        # condição de comparação
        
        if row['EXISTE']:
            arquivo = row['ARQUIVO']
            pasta_destino = row['PASTA_DESTINO']
            
            # Verifica existencia de linha com CERTIDAO_ID, CLIENTE_ID e ARQUIVO iguais
            condicao_existente = (
                (df_monitor['CERTIDAO_ID'] == certidao_id) &
                (df_monitor['CLIENTE_ID'] == cliente_id) &
                (df_monitor['ARQUIVO'] == arquivo)
            )
            
            if df_monitor[condicao_existente].empty:
                novo_dado = {
                    'CERTIDAO_ID': certidao_id,
                    'CLIENTE_ID': cliente_id,
                    'PASTA_DESTINO': pasta_destino,
                    'ARQUIVO': arquivo,
                    'STATUS': 'adicionado',
                    'DATA_INICIO': None,
                    'DATA_VENCIMENTO': None,
                    'NUMERO_CERTIDAO': None,
                    'NADA_CONSTA': None,
                    'CREATED_AT': None,
                    'UPDATED_AT': horario_atual
                }
                certidao_adicionada.append(novo_dado)
                df_monitor = pd.concat([df_monitor, pd.DataFrame([novo_dado])], ignore_index=True)

        # Condição para quando 'EXISTE' é False
        else:
            condicao_existente = (
                (df_monitor['CERTIDAO_ID'] == certidao_id) & 
                (df_monitor['CLIENTE_ID'] == cliente_id) & 
                (df_monitor['ARQUIVO'] == row['ARQUIVO'])
                )
            if not df_monitor[condicao_existente].empty and row['STATUS'] in ['monitorando', 'VENCIDO']:
                df_monitor.loc[condicao_existente, 'STATUS'] = 'ERROR'
                # registrar_erro('arquivos_novos_dbmonitor', {
                #     'CERTIDAO_ID': certidao_id,
                #     'CLIENTE_ID': cliente_id,
                #     'ARQUIVO': row['ARQUIVO'] })

            certidao_nao_encontrada.append({
                'CERTIDAO_ID': certidao_id,
                'CLIENTE_ID': cliente_id,
                'PASTA_DESTINO': row['PASTA_DESTINO'] if pd.notnull(row['PASTA_DESTINO']) else None,
                'TIME_STAMP': horario_atual
            })

    # Exibe o conteúdo atualizado de df_monitor e a lista de certidões não encontradas
    print(f'Certidões não encontradas:{len(certidao_nao_encontrada)}')
    print(certidao_nao_encontrada)
    print('df_monitor atualizado')
    print(f'Certidões adicionadas:{len(certidao_adicionada)}')

    return df_monitor

In [179]:
df_monitor_final = arquivos_novos_dbmonitor(df_monitor, df_cliente_certidao_resultado)

Certidões não encontradas:6
[{'CERTIDAO_ID': 'TESTE-TESTE', 'CLIENTE_ID': 'AUTOLTDA', 'PASTA_DESTINO': 'clientes/autodata/', 'TIME_STAMP': datetime.datetime(2024, 11, 16, 14, 55, 5)}, {'CERTIDAO_ID': 'GDF-CND', 'CLIENTE_ID': 'BIGBOX', 'PASTA_DESTINO': 'clientes/bigbox/', 'TIME_STAMP': datetime.datetime(2024, 11, 16, 14, 55, 5)}, {'CERTIDAO_ID': 'TESTE-TESTE', 'CLIENTE_ID': 'BIGBOX', 'PASTA_DESTINO': 'clientes/bigbox/', 'TIME_STAMP': datetime.datetime(2024, 11, 16, 14, 55, 5)}, {'CERTIDAO_ID': 'TESTE-TESTE', 'CLIENTE_ID': 'BOTMEI', 'PASTA_DESTINO': 'clientes/botmei/', 'TIME_STAMP': datetime.datetime(2024, 11, 16, 14, 55, 5)}, {'CERTIDAO_ID': 'TESTE-TESTE', 'CLIENTE_ID': 'DROSARIO', 'PASTA_DESTINO': 'clientes/drosario/', 'TIME_STAMP': datetime.datetime(2024, 11, 16, 14, 55, 5)}, {'CERTIDAO_ID': 'TESTE-TESTE', 'CLIENTE_ID': 'TLTDA', 'PASTA_DESTINO': 'clientes/testeltda/', 'TIME_STAMP': datetime.datetime(2024, 11, 16, 14, 55, 5)}]
df_monitor atualizado
Certidões adicionadas:39


C:\Users\User\AppData\Local\Temp\ipykernel_8016\2968264422.py:41: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_monitor = pd.concat([df_monitor, pd.DataFrame([novo_dado])], ignore_index=True)


### atualizar_planilha()
salva dataframe na planilha db_monitor

In [180]:
def atualizar_planilha(gestao_monitoramento, workbook, df_monitor):
    # Carrega o arquivo Excel
    workbook = load_workbook(gestao_monitoramento)
    
    # Seleciona a aba 'db_monitor'
    sheet = workbook['db_monitor']

    # Localiza a tabela 'fDB_monitor'
    table = None
    for tbl in sheet._tables.values():
        if tbl.name == 'fDB_monitor':
            table = tbl
            break

    if table is None:
        print("Tabela 'fDB_monitor' não encontrada.")
        return

    # Limpa todas as linhas da tabela (exceto o cabeçalho)
    start_row = table.ref.split(':')[0][1:]
    end_row = table.ref.split(':')[1][1:]
    start_col = table.ref.split(':')[0][0]
    end_col = table.ref.split(':')[1][0]

    for row in sheet.iter_rows(min_row=int(start_row) + 1, max_row=int(end_row), min_col=ord(start_col) - 64, max_col=ord(end_col) - 64):
        for cell in row:
            cell.value = None

    # Preenche os dados do DataFrame a partir da linha da tabela
    for index, row in df_monitor.iterrows():
        for col_num, value in enumerate(row, start=1):  # start=1 para a primeira coluna (A)
            sheet.cell(row=int(start_row) + 1 + index, column=col_num, value=value)  # Inicia na linha da tabela, logo abaixo do cabeçalho

    # Atualiza o intervalo da tabela para incluir todas as novas linhas
    table.ref = f"{start_col}{start_row}:{end_col}{int(start_row) + len(df_monitor)}"

    # Salva as alterações no arquivo Excel
    workbook.save(gestao_monitoramento)
    print("Arquivo Excel atualizado com os dados do DataFrame!")

# Exemplo de uso
gestao_monitoramento = 'gestao_monitoramento.xlsx'
workbook = load_workbook(gestao_monitoramento)
df_monitor = pd.read_excel(gestao_monitoramento, sheet_name='db_monitor')

atualizar_planilha(gestao_monitoramento, workbook, df_monitor_final)

Arquivo Excel atualizado com os dados do DataFrame!


### ERRO

In [155]:
def registrar_erro(funcao, dados):
    global log_erros 
    log_erros.append({ 
        'funcao': funcao, 
        'dados': dados, 
        'timestamp': datetime.now().replace(microsecond=0)})

## Funções extração

In [188]:
df_monitor = pd.read_excel(gestao_monitoramento, sheet_name='db_monitor')

pdf_path = 'clientes/drosario/CN Impropidade Administrativa e Inelegibilidade.pdf'

df_monitor.info()
df_monitor.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   CERTIDAO_ID      39 non-null     object        
 1   CLIENTE_ID       39 non-null     object        
 2   PASTA_DESTINO    39 non-null     object        
 3   ARQUIVO          39 non-null     object        
 4   STATUS           39 non-null     object        
 5   DATA_INICIO      0 non-null      float64       
 6   DATA_VENCIMENTO  0 non-null      float64       
 7   NUMERO_CERTIDAO  0 non-null      float64       
 8   NADA_CONSTA      0 non-null      float64       
 9   CREATED_AT       0 non-null      float64       
 10  UPDATED_AT       39 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(5), object(5)
memory usage: 3.5+ KB


,CERTIDAO_ID,CLIENTE_ID,PASTA_DESTINO,ARQUIVO,STATUS,DATA_INICIO,DATA_VENCIMENTO,NUMERO_CERTIDAO,NADA_CONSTA,CREATED_AT,UPDATED_AT
0,CEF-CRFGTS,AUTOLTDA,clientes/autodata/,2024-09-30_CR FGTS.pdf,adicionado,NaN,NaN,NaN,NaN,NaN,2024-11-16 14:55:05
1,TJDFT-CNFR,AUTOLTDA,clientes/autodata/,2024-09-09_CN Falencia e Recuperacao.pdf,adicionado,NaN,NaN,NaN,NaN,NaN,2024-11-16 14:55:05
2,CGU-CNC,AUTOLTDA,clientes/autodata/,"2024-09-13_CN Correcional (ePAD, CGU-PJ, CEIS,...",adicionado,NaN,NaN,NaN,NaN,NaN,2024-11-16 14:55:05
3,FN-CNDTF,AUTOLTDA,clientes/autodata/,2024-09-06_CND Tributos Federais.pdf,adicionado,NaN,NaN,NaN,NaN,NaN,2024-11-16 14:55:05
4,GDF-CND,AUTOLTDA,clientes/autodata/,2024-09-20_CND GDF.pdf,adicionado,NaN,NaN,NaN,NaN,NaN,2024-11-16 14:55:05


### data_criacao_pdf(pdf_path)

In [189]:
def data_criacao_pdf(pdf_path):
    try:
        timestamp =  os.path.getctime(pdf_path)
        created_at = datetime.fromtimestamp(timestamp).replace(microsecond=0)#.strftime('%Y-%m-%d %H:%M:%S')
        return created_at
    except Exception as e:
        print(f'Erro ao obter a data de criação do arquivo: {e}')
        return None
    
data_criacao_pdf(pdf_path)

datetime.datetime(2024, 11, 16, 2, 51, 45)

### extrair_data_inicio(texto_pdf)

In [204]:
def extrair_data_inicio(texto_pdf):
    try:
        padrao_regex = [
            r'Validade:(\d{2}/\d{2}/\d{4})',
            r'Emitida gratuitamente pela internet em:\s*(\d{2}/\d{2}/\d{4})',
            r'Gerado em:\s*(\d{2}/\d{2}/\d{4})',
            r'Certidão emitida às\s*\d{2}:\d{2}:\d{2}\s*do dia\s*(\d{2}/\d{2}/\d{4})',
            r'Emitida às\s*\d{2}:\d{2}:\d{2}\s*do dia\s*(\d{2}/\d{2}/\d{4})',
            r'Expedição:\s*(\d{2}/\d{2}/\d{4})\s*, às\s*\d{2}:\d{2}:\d{2}',
            r'Este documento foi emitido no dia\s*(\d{2}/\d{2}/\d{4})',
            r'Certidão emitida via internet em\s*(\d{2}/\d{2}/\d{4})\s*às\s*\d{2}:\d{2}:\d{2}'
        ]

        for padrao in padrao_regex:
            par = re.search(padrao, texto_pdf)
            if par:
                data_encontrada = par.group(1)
                data_objeto = datetime.strptime(data_encontrada, '%d/%m/%Y')
                # data_formatada = data_objeto.strftime('%Y-%m-%d')
                return data_objeto

        print('Data de início não encontrada.')
        return None

    except Exception as e:
        print(f'Erro ao extrair data de início do arquivo: {e}')
        return None
    

with pdfplumber.open(pdf_path) as pdf:
                    texto_pdf = ''.join(page.extract_text() or '' for page in pdf.pages)
# texto_pdf = None
extrair_data_inicio(texto_pdf)

datetime.datetime(2024, 11, 14, 0, 0)

### extrair_numero_certidao(texto_pdf, row)

In [ ]:
def extrair_numero_certidao(texto_pdf, row):
    try:
        padrao_regex = [
            r'Certificação Número:\s*([\w./]+)',  # Ex: "Certificação Número: 024102105285887458243"
            r'Selo digital de segurança:\s*([\w./]+)',  # Ex: "Selo digital de segurança: 2024.CTD.ZZQ3.G1NW.0B85.GEDK.UR8P"
            r'número de controle\s*\n*([\w./]+)',  # Ex: "número de controle\n672D.DF85.D1C3.5589"
            r'Código de controle da certidão:\s*([\w./]+)',  # Ex: "Código de controle da certidão: YLkevKTcTH7aFG1S8seQ"
            r'Certidão nº:\s*([\w./]+)',  # Ex: "Certidão nº: 76418628/2024"
            r'CERTIDÃO Nº:\s*([\w./]+)'
        ]
        if row['CERTIDAO_ID'] == 'GDF-DIF':
            return '-não existe-'
        else:
            for padrao in padrao_regex:
                par = re.search(padrao, texto_pdf)
                if par:
                    numero_certidao = par.group(1)
                    return numero_certidao


    except Exception as e:
        print(f'Erro ao extrair número da certidão: {e}')
        return None

### extrair_nada_consta(texto_pdf)

In [206]:
def extrair_nada_consta(texto_pdf):
    try:
        padrao_regex = [
            r'NÃO\s*CONSTA',                    # "NÃO CONSTA"
            r'NADA\s*CONSTA',                   # "NADA CONSTA"
            r'encontra-se em situação regular', # "encontra-se em situação regular"
            r'não\s*constam',                   # "não constam"
            r'NÃO\s*CONSTAM',                   # "NÃO CONSTAM"
            r'ATIVO\s*\d{2}/\d{2}/\d{4}',       # Ativo - verifica cadastro ativo
            r'tem\s+os\s+mesmos\s+efeitos\s+da\s+certidão\s+negativa\.', # positiva com efeito de negativa
            r'certidão\s+positiva\s+de\s+débitos\s+com\s+efeito\s+de\s+negativa',
            r'certidão\s+positiva\s+com\s+efeito\s+de\s+negativa'
        ]

        for padrao in padrao_regex:
            par = re.search(padrao, texto_pdf)
            if par:
                return True
        
        return False
    
    except Exception as e:
        print(f'Erro ao extrair Nada Consta da certidão: {e}')
        return False

### extrair_data_vencimento(texto_pdf, data_inicio, row)

In [252]:
def extrair_data_vencimento(texto_pdf, row):
    data_inicio = row['DATA_INICIO']
    try:
        padrao_regex = [
            r"Validade:\s*\d{2}/\d{2}/\d{4}\s*a\s*(\d{2}/\d{2}/\d{4})",  # Ex: "Validade:21/10/2024 a 19/11/2024" (pegar a segunda data)
            r"Validade:\s*(\d{2}/\d{2}/\d{4})\s*",  # Ex: "Validade: 03/05/2025 -"
            r"Válida até\s*(\d{2}/\d{2}/\d{4})",  # Ex: "Válida até 14/04/2025"
            r"Válida até\s*(\d{2}) de (\w+) de (\d{4})",  # Ex: "Válida até 02 de fevereiro de 2025"
            r", com validade até o dia\s*(\d{2}/\d{2}/\d{4})"  # Ex: ", com validade até o dia 04/12/2024"
        ]
        
        meses = {
            'janeiro': '01', 'fevereiro': '02', 'março': '03', 'abril': '04',
            'maio': '05', 'junho': '06', 'julho': '07', 'agosto': '08',
            'setembro': '09', 'outubro': '10', 'novembro': '11', 'dezembro': '12'
        }

        for padrao in padrao_regex:
            if row['CERTIDAO_ID'] in ('GDF-DIF','TJDFT-CNFR','CNJ-CNIADM'):
                print('TENTANDDO NOVA DATA AQUI')
                # data_objeto = datetime.strptime(data_inicio, '%Y-%m-%d')
                data_vencimento = data_inicio + timedelta(days=30)
                print(f'data_vencimento: {data_vencimento}')
                return data_vencimento
            
            par = re.search(padrao, texto_pdf)
            if par:
                # data por extenso
                if len(par.groups()) == 3:
                    dia, mes_extenso, ano = par.groups()
                    mes = meses.get(mes_extenso.lower()) # pega no dicionario
                    data_str = (f'{ano}-{mes}-{dia}')
                    data = datetime.strptime(data_str, '%Y-%m-%d')
                    return data
                else:
                    data_encontrada = par.group(1)
                    data_objeto = datetime.strptime(data_encontrada, '%d/%m/%Y')
                    # data_formatada = data_objeto.strftime('%Y-%m-%d')
                    return data_objeto  # Retorne a data encontrada
    except Exception as e:
        print(f'Erro ao extrair data de vencimento da certidão: {e}')
        return False

### renomear_pdf_path(data_vencimento, certidao_id, pasta_destino, nome_arquivo_monitor, df_cert_config)

In [ ]:
# def renomear_pdf_path(data_vencimento, certidao_id, pasta_destino, nome_arquivo_monitor, df_cert_config):
    
#     # Capturar a linha onde 'NOME_ARQUIVO' é igual a 'certidao_id'
#     linha_no_cert_config = df_cert_config.loc[df_cert_config['CERTIDAO_ID'] == certidao_id]

#     if not linha_no_cert_config.empty:
#             # nome referencia
#             nome_ref = linha_no_cert_config['NOME_ARQUIVO'].values[0]
            
#             arquivos_na_pasta = os.listdir(pasta_destino)

#             for arquivo in arquivos_na_pasta:
#                 if nome_arquivo_monitor in arquivo:

#                     indice_nome = nome_arquivo_monitor.find(nome_ref)
#                     if indice_nome != -1:
#                         print(f'nome original: {nome_arquivo_monitor}')
#                         nome_recortado = nome_arquivo_monitor[indice_nome:]
#                         print("Resultado:", nome_recortado)
                        
#                         # novo nome = data de vencimento e nome recortado
#                         data_vencimento_str = data_vencimento.strftime('%Y-%m-%d')
#                         novo_nome = f"{data_vencimento_str}_{nome_recortado}"
#                         novo_path = os.path.join(pasta_destino, novo_nome)
                        
#                         # Caminho do arquivo antigo
#                         arquivo_path = os.path.join(pasta_destino, nome_arquivo_monitor)
                        
#                         # Verifica se o novo nome já existe na pasta, incrementa o contador se necessário
#                         contador = 1
#                         while os.path.exists(novo_path):
#                             # Se o arquivo já existe, divide o nome e extensão e adiciona o contador
#                             novo_nome = f'{data_vencimento_str}({contador})_{nome_recortado}'
#                             novo_path = os.path.join(pasta_destino, novo_nome)
#                             print(novo_path)
#                             contador += 1

#                         # Renomeia o arquivo antigo para o novo nome
#                         try:
#                             os.rename(arquivo_path, novo_path)
#                             print(f"Arquivo renomeado para: {novo_path}")
#                             return novo_nome
#                         except FileNotFoundError:
#                             print(f"Arquivo original '{arquivo_path}' não encontrado.")
#                         except Exception as e:
#                             print(f"Erro ao renomear o arquivo: {e}")

In [ ]:
import os
from datetime import datetime

def renomear_pdf_path(data_vencimento, certidao_id, pasta_destino, nome_arquivo_monitor, df_cert_config, status, cliente_id):
    """
    Renomeia ou move arquivos PDF com base no status ('adicionado', 'monitorando', 'vencido').

    Args:
        data_vencimento (datetime): Data de vencimento.
        certidao_id (str): Identificador da certidão.
        pasta_destino (str): Pasta onde os arquivos estão localizados.
        nome_arquivo_monitor (str): Nome base do arquivo monitorado.
        df_cert_config (DataFrame): Configurações da certidão.
        status (str): Pode ser 'adicionado', 'monitorando' ou 'vencido'.
        cliente_id (str, optional): Identificador do cliente (necessário para 'adicionado').

    Returns:
        str: Nome do arquivo atualizado ou mensagem indicando nenhuma ação realizada.
    """
    # Filtrar a linha no DataFrame
    linha_no_cert_config = df_cert_config.loc[df_cert_config['CERTIDAO_ID'] == certidao_id]

    if linha_no_cert_config.empty:
        print("Certidão não encontrada no DataFrame.")
        return None

    nome_ref = linha_no_cert_config['NOME_ARQUIVO'].values[0]
    arquivos_na_pasta = os.listdir(pasta_destino)

    for arquivo in arquivos_na_pasta:
        if nome_arquivo_monitor in arquivo:
            arquivo_path = os.path.join(pasta_destino, arquivo)

            # Status: Adicionado
            if status == 'adicionado':
                indice_nome = nome_arquivo_monitor.find(nome_ref)
                if indice_nome != -1:
                    nome_recortado = nome_arquivo_monitor[indice_nome:]
                    data_vencimento_str = data_vencimento.strftime('%Y-%m-%d')
                    novo_nome = f"{data_vencimento_str}_{cliente_id}_{certidao_id}_{nome_recortado}"
                    novo_path = os.path.join(pasta_destino, novo_nome)

                    # Evitar sobrescrever arquivos existentes
                    contador = 1
                    while os.path.exists(novo_path):
                        novo_nome = f"{data_vencimento_str}_{cliente_id}_{certidao_id}_{nome_recortado}({contador})"
                        novo_path = os.path.join(pasta_destino, novo_nome)
                        contador += 1

                    # Renomear o arquivo
                    try:
                        os.rename(arquivo_path, novo_path)
                        print(f"Arquivo renomeado para: {novo_path}")
                        return novo_nome
                    except Exception as e:
                        print(f"Erro ao renomear arquivo: {e}")
                        return None

            # Status: Monitorando
            elif status == 'monitorando':
                if nome_arquivo_monitor == arquivo:
                    print(f"O arquivo {arquivo} já está sendo monitorado. Nenhuma ação necessária.")
                    return "Nenhuma ação necessária"

            # Status: Vencido
            elif status == 'vencido':
                data_hora_atual = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
                nome_vencido = f"{os.path.splitext(arquivo)[0]}_b-{data_hora_atual}.pdf"
                nova_pasta = os.path.join(pasta_destino, "certidoes_vencidas")

                # Criar a pasta de vencidos, se não existir
                if not os.path.exists(nova_pasta):
                    os.makedirs(nova_pasta)

                novo_path = os.path.join(nova_pasta, nome_vencido)

                # Mover o arquivo para a pasta de vencidos
                try:
                    os.rename(arquivo_path, novo_path)
                    print(f"Arquivo vencido movido para: {novo_path}")
                    return nome_vencido
                except Exception as e:
                    print(f"Erro ao mover arquivo vencido: {e}")
                    return None

    print(f"Nenhum arquivo correspondente encontrado para {nome_arquivo_monitor}.")
    return None


#### novo rename e processa_monitor

In [240]:
df_monitor.columns

Index(['CERTIDAO_ID', 'CLIENTE_ID', 'PASTA_DESTINO', 'ARQUIVO', 'STATUS',
       'DATA_INICIO', 'DATA_VENCIMENTO', 'NUMERO_CERTIDAO', 'NADA_CONSTA',
       'CREATED_AT', 'UPDATED_AT'],
      dtype='object')

In [281]:
def renomear_pdf_path(row):
    print('iniciando renomear_pdf_patch()')
    # Carrega df_cert_config e filtra linha
    df_cert_config = pd.read_excel(gestao_monitoramento, sheet_name='config_certidoes', index_col=None)
    linha_certidao_id = df_cert_config.loc[df_cert_config['CERTIDAO_ID'] == row['CERTIDAO_ID']]

    if linha_certidao_id.empty:
        print(f"Certidão {row['CERTIDAO_ID']} não encontrada no config_certidoes.")
        return None
    
    nome_ref = linha_certidao_id['NOME_ARQUIVO'].values[0]
    arquivos_na_pasta = os.listdir(row['PASTA_DESTINO'])

    print(f'nome arquivo_monitor: {row["ARQUIVO"]}')
    for arquivo in arquivos_na_pasta:
        if row['ARQUIVO'] == arquivo:
            print(f'nome arquivo na pasta: {arquivo}')
            indice_nome = row['ARQUIVO'].find(nome_ref)
            if indice_nome != -1:
                    nome_recortado = row['ARQUIVO'][indice_nome:]
                    print(f'nome recortado {nome_recortado}')
                    data_vencimento_str = row['DATA_VENCIMENTO'].strftime('%Y-%m-%d')
                    novo_nome = f"{data_vencimento_str}_({row['CLIENTE_ID']}_{row['CERTIDAO_ID']})_{nome_recortado}"
                    novo_path = os.path.join(row["PASTA_DESTINO"], novo_nome)
                    print(novo_nome)

                    if novo_nome != row['ARQUIVO']:
                         print('O nome é diferente!')
            
    print()

    




In [282]:
gestao_monitoramento = 'gestao_monitoramento.xlsx'
# salva formatação
workbook = load_workbook(gestao_monitoramento)
df_clientes = pd.read_excel(gestao_monitoramento, sheet_name='config_clientes', index_col=None)
df_cert_config = pd.read_excel(gestao_monitoramento, sheet_name='config_certidoes', index_col=None)
df_monitor = pd.read_excel(gestao_monitoramento, sheet_name='db_monitor')

## PROCESSA ADICIONADOS
def processa_adicionados(row):

    pdf_path = os.path.join(row['PASTA_DESTINO'], row['ARQUIVO'])

    with pdfplumber.open(pdf_path) as pdf:
        texto_pdf = ''.join(page.extract_text() or '' for page in pdf.pages)

        row['CREATED_AT'] = data_criacao_pdf(pdf_path)

        row['DATA_INICIO'] = extrair_data_inicio(texto_pdf)

        row['NUMERO_CERTIDAO'] = extrair_numero_certidao(texto_pdf, row)

        row['NADA_CONSTA'] = extrair_nada_consta(texto_pdf)

        row['DATA_VENCIMENTO'] = extrair_data_vencimento(texto_pdf, row)

        renomear_pdf_path(row)

    return row


for index, row in df_monitor.iterrows():
    if row['STATUS'] == 'adicionado':
        # processa e atualiza a row
        df_monitor.iloc[index] = processa_adicionados(row)

    # elif row['STATUS'] == 'monitorando':
    #     print('Olá')
    # elif row['STATUS'] == 'VENCIDO':
    #     print('Tchau')
    # print(row)

iniciando renomear_pdf_patch()
nome arquivo_monitor: 2024-11-14_CND Trabalhista.pdf
nome arquivo na pasta: 2024-11-14_CND Trabalhista.pdf
nome recortado CND Trabalhista.pdf
2024-11-14_(AUTOLTDA_TST-CNDT)_CND Trabalhista.pdf
O nome é diferente!

TENTANDDO NOVA DATA AQUI
data_vencimento: 2024-10-25 00:00:00
iniciando renomear_pdf_patch()
nome arquivo_monitor: 2024-10-25_CN Impropidade Administrativa e Inelegibilidade.pdf
nome arquivo na pasta: 2024-10-25_CN Impropidade Administrativa e Inelegibilidade.pdf
nome recortado CN Impropidade Administrativa e Inelegibilidade.pdf
2024-10-25_(AUTOLTDA_CNJ-CNIADM)_CN Impropidade Administrativa e Inelegibilidade.pdf
O nome é diferente!



C:\Users\User\AppData\Local\Temp\ipykernel_8016\2002094560.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_monitor.iloc[index] = processa_adicionados(row)


iniciando renomear_pdf_patch()
nome arquivo_monitor: 2024-12-07_CR FGTS.pdf
nome arquivo na pasta: 2024-12-07_CR FGTS.pdf
nome recortado CR FGTS.pdf
2024-12-07_(BIGBOX_CEF-CRFGTS)_CR FGTS.pdf
O nome é diferente!

TENTANDDO NOVA DATA AQUI
data_vencimento: 2024-12-14 00:00:00
iniciando renomear_pdf_patch()
nome arquivo_monitor: 2024-12-14_CN Falencia e Recuperacao.pdf
nome arquivo na pasta: 2024-12-14_CN Falencia e Recuperacao.pdf
nome recortado CN Falencia e Recuperacao.pdf
2024-12-14_(BIGBOX_TJDFT-CNFR)_CN Falencia e Recuperacao.pdf
O nome é diferente!

iniciando renomear_pdf_patch()
nome arquivo_monitor: 2024-12-14_CN Correcional (ePAD, CGU-PJ, CEIS, CNEP e CEPIM).pdf
nome arquivo na pasta: 2024-12-14_CN Correcional (ePAD, CGU-PJ, CEIS, CNEP e CEPIM).pdf
nome recortado CN Correcional (ePAD, CGU-PJ, CEIS, CNEP e CEPIM).pdf
2024-12-14_(BIGBOX_CGU-CNC)_CN Correcional (ePAD, CGU-PJ, CEIS, CNEP e CEPIM).pdf
O nome é diferente!

iniciando renomear_pdf_patch()
nome arquivo_monitor: 2025-04-1

In [257]:
for index, row in df_monitor.iterrows():
    if row['STATUS'] == 'adicionado':
        print('OI')
        print(f'CREATED_AT {row["CREATED_AT"]}')
        print(f'DATA_INICIO {row["DATA_INICIO"]}')
        print(f'NADA_CONSTA {row["NADA_CONSTA"]}')
        print(f'DATA_VENCIMENTO {row["DATA_VENCIMENTO"]}')

OI
CREATED_AT 2024-11-16 02:51:45
DATA_INICIO 2024-10-02 00:00:00
NADA_CONSTA True
DATA_VENCIMENTO 2024-11-14 00:00:00
OI
CREATED_AT 2024-11-16 02:51:45
DATA_INICIO 2024-09-25 00:00:00
NADA_CONSTA True
DATA_VENCIMENTO 2024-10-25 00:00:00
OI
CREATED_AT 2024-11-16 02:51:45
DATA_INICIO 2024-11-08 00:00:00
NADA_CONSTA True
DATA_VENCIMENTO 2024-12-07 00:00:00
OI
CREATED_AT 2024-11-16 02:51:45
DATA_INICIO 2024-11-14 00:00:00
NADA_CONSTA True
DATA_VENCIMENTO 2024-12-14 00:00:00
OI
CREATED_AT 2024-11-16 02:51:45
DATA_INICIO 2024-11-14 00:00:00
NADA_CONSTA True
DATA_VENCIMENTO 2024-12-14 00:00:00
OI
CREATED_AT 2024-11-16 02:51:45
DATA_INICIO 2024-10-16 00:00:00
NADA_CONSTA True
DATA_VENCIMENTO 2025-04-14 00:00:00
OI
CREATED_AT 2024-11-16 02:51:45
DATA_INICIO 2024-11-14 00:00:00
NADA_CONSTA True
DATA_VENCIMENTO 2024-12-14 00:00:00
OI
CREATED_AT 2024-11-16 02:51:45
DATA_INICIO 2024-11-14 00:00:00
NADA_CONSTA True
DATA_VENCIMENTO 2025-05-13 00:00:00
OI
CREATED_AT 2024-11-16 02:51:45
DATA_INICIO 20

### processa_monitor(df_monitor)

In [225]:
def processa_monitor(df_monitor):
    horario_atual = datetime.now()

    # Itera no DF 
    for index, row in df_monitor.iterrows():
        if row['STATUS'] == 'adicionado':
            pdf_path = os.path.join(row['PASTA_DESTINO'], row['ARQUIVO'])
            print(pdf_path)

            # Inicializa variáveis para cada coluna que pode ser preenchida
            cliente_id = ['CLIENTE_ID']
            status = row['STATUS']
            pasta_destino = row['PASTA_DESTINO']
            certidao_id = row['CERTIDAO_ID']
            nome_arquivo_monitor = row['ARQUIVO']
            created_at = row['CREATED_AT']
            data_inicio = row['DATA_INICIO']
            numero_certidao = row['NUMERO_CERTIDAO']
            nada_consta = row['NADA_CONSTA']
            data_vencimento = row['DATA_VENCIMENTO']

            # Flag de erro para controlar a execução geral
            error_flag = 0  # Começa com 0, indicando que tudo está bem

            try:
                # Abre o PDF e extrai o texto
                with pdfplumber.open(pdf_path) as pdf:
                    texto_pdf = ''.join(page.extract_text() or '' for page in pdf.pages)

                # Verifica e preenche cada coluna individualmente se estiver vazia

                # DATA DE CRIAÇÃO
                if pd.isna(row['CREATED_AT']) or row['CREATED_AT'] == 'nan':
                    try:
                        created_at = data_criacao_pdf(pdf_path)
                        print(f'created_at: {created_at}')
                        df_monitor.at[index, 'CREATED_AT'] = created_at

                    except Exception as e:
                        print(f"Erro ao extrair created_at: {e}")

                # DATA DE INÍCIO
                if pd.isna(row['DATA_INICIO']) or row['DATA_INICIO'] == 'nan':
                    try:
                        data_inicio = extrair_data_inicio(texto_pdf)
                        print(f'data_inicio: {data_inicio}')
                        df_monitor.at[index, 'DATA_INICIO'] = data_inicio

                    except Exception as e:
                        print(f"Erro ao extrair data_inicio: {e}")
                
                # NÚMERO DA CERTIDÃO
                if pd.isna(row['NUMERO_CERTIDAO']) or row['NUMERO_CERTIDAO'] == 'nan':
                    try:
                        numero_certidao = extrair_numero_certidao(texto_pdf, row)
                        print(f'numero_certidao: {numero_certidao}')
                    except Exception as e:
                        print(f"Erro ao extrair numero_certidao: {e}")

                # NADA CONSTA
                if pd.isna(row['NADA_CONSTA']) or row['NADA_CONSTA'] == 'nan':
                    try:
                        nada_consta = extrair_nada_consta(texto_pdf)
                        print(f'nada_consta: {nada_consta}')
                    except Exception as e:
                        print(f"Erro ao extrair nada_consta: {e}")

                # DATA DE VENCIMENTO
                if pd.isna(row['DATA_VENCIMENTO']) or row['DATA_VENCIMENTO'] == 'nan':
                    try:
                        data_vencimento = extrair_data_vencimento(texto_pdf, data_inicio, row)
                        print(f'data_vencimento: {data_vencimento}')
                    except Exception as e:
                        print(f"Erro ao extrair data_vencimento: {e}")

                # RENOMEANDO ARQUIVO
                novo_nome = renomear_pdf_path(data_vencimento, certidao_id, pasta_destino, nome_arquivo_monitor, df_cert_config, status, cliente_id)



            except Exception as e:
                print(f"Erro ao processar o arquivo {pdf_path}: {e}")

            finally:
                df_monitor.at[index, 'STATUS'] = 'monitorando'
                df_monitor.at[index, 'DATA_VENCIMENTO'] = data_vencimento
                df_monitor.at[index, 'NUMERO_CERTIDAO'] = numero_certidao
                df_monitor.at[index, 'NADA_CONSTA'] = nada_consta
                df_monitor.at[index, 'UPDATED_AT'] = horario_atual
                if novo_nome == None:
                    continue
                else:
                    df_monitor.at[index, 'ARQUIVO'] = novo_nome
                
                # if error_flag == 1:
                #     print('error flag 1')
                # else:
                #     print(f"Atualização parcial realizada para o index {index}, devido ao erro_flag.")

    return df_monitor
    

df_monitor = processa_monitor(df_monitor)

df_monitor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   CERTIDAO_ID      39 non-null     object        
 1   CLIENTE_ID       39 non-null     object        
 2   PASTA_DESTINO    39 non-null     object        
 3   ARQUIVO          39 non-null     object        
 4   STATUS           39 non-null     object        
 5   DATA_INICIO      39 non-null     datetime64[ns]
 6   DATA_VENCIMENTO  39 non-null     datetime64[ns]
 7   NUMERO_CERTIDAO  39 non-null     object        
 8   NADA_CONSTA      39 non-null     bool          
 9   CREATED_AT       39 non-null     datetime64[ns]
 10  UPDATED_AT       39 non-null     datetime64[ns]
dtypes: bool(1), datetime64[ns](4), object(6)
memory usage: 3.2+ KB


In [ ]:
df_monitor

In [219]:
import os
from datetime import datetime

def renomear_pdf_path(data_vencimento, certidao_id, pasta_destino, nome_arquivo_monitor, df_cert_config, status, cliente_id=None):
    """
    Renomeia ou move arquivos PDF com base no status ('adicionado', 'monitorando', 'vencido').

    Args:
        data_vencimento (datetime): Data de vencimento.
        certidao_id (str): Identificador da certidão.
        pasta_destino (str): Pasta onde os arquivos estão localizados.
        nome_arquivo_monitor (str): Nome base do arquivo monitorado.
        df_cert_config (DataFrame): Configurações da certidão.
        status (str): Pode ser 'adicionado', 'monitorando' ou 'vencido'.
        cliente_id (str, optional): Identificador do cliente (necessário para 'adicionado').

    Returns:
        str: Nome do arquivo atualizado ou mensagem indicando nenhuma ação realizada.
    """
    # Filtrar a linha no DataFrame
    linha_no_cert_config = df_cert_config.loc[df_cert_config['CERTIDAO_ID'] == certidao_id]

    if linha_no_cert_config.empty:
        print("Certidão não encontrada no DataFrame.")
        return None

    nome_ref = linha_no_cert_config['NOME_ARQUIVO'].values[0]
    arquivos_na_pasta = os.listdir(pasta_destino)

    for arquivo in arquivos_na_pasta:
        if nome_arquivo_monitor in arquivo:
            arquivo_path = os.path.join(pasta_destino, arquivo)

            # Status: Adicionado
            if status == 'adicionado':
                indice_nome = nome_arquivo_monitor.find(nome_ref)
                if indice_nome != -1:
                    nome_recortado = nome_arquivo_monitor[indice_nome:]
                    data_vencimento_str = data_vencimento.strftime('%Y-%m-%d')
                    novo_nome = f"{data_vencimento_str}_{cliente_id}_{certidao_id}_{nome_recortado}"
                    novo_path = os.path.join(pasta_destino, novo_nome)

                    # Evitar sobrescrever arquivos existentes
                    contador = 1
                    while os.path.exists(novo_path):
                        novo_nome = f"{data_vencimento_str}_{cliente_id}_{certidao_id}_{nome_recortado}({contador})"
                        novo_path = os.path.join(pasta_destino, novo_nome)
                        contador += 1

                    # Renomear o arquivo
                    try:
                        os.rename(arquivo_path, novo_path)
                        print(f"Arquivo renomeado para: {novo_path}")
                        return novo_nome
                    except Exception as e:
                        print(f"Erro ao renomear arquivo: {e}")
                        return None

            # Status: Monitorando
            elif status == 'monitorando':
                if nome_arquivo_monitor == arquivo:
                    print(f"O arquivo {arquivo} já está sendo monitorado. Nenhuma ação necessária.")
                    return "Nenhuma ação necessária"

            # Status: Vencido
            elif status == 'vencido':
                data_hora_atual = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
                nome_vencido = f"{os.path.splitext(arquivo)[0]}_b-{data_hora_atual}.pdf"
                nova_pasta = os.path.join(pasta_destino, "certidoes_vencidas")

                # Criar a pasta de vencidos, se não existir
                if not os.path.exists(nova_pasta):
                    os.makedirs(nova_pasta)

                novo_path = os.path.join(nova_pasta, nome_vencido)

                # Mover o arquivo para a pasta de vencidos
                try:
                    os.rename(arquivo_path, novo_path)
                    print(f"Arquivo vencido movido para: {novo_path}")
                    return nome_vencido
                except Exception as e:
                    print(f"Erro ao mover arquivo vencido: {e}")
                    return None

    print(f"Nenhum arquivo correspondente encontrado para {nome_arquivo_monitor}.")
    return None


In [220]:
df_monitor['NADA_CONSTA'] = df_monitor['NADA_CONSTA'].astype(bool)
df_monitor.info()
df_monitor.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   CERTIDAO_ID      39 non-null     object        
 1   CLIENTE_ID       39 non-null     object        
 2   PASTA_DESTINO    39 non-null     object        
 3   ARQUIVO          39 non-null     object        
 4   STATUS           39 non-null     object        
 5   DATA_INICIO      39 non-null     datetime64[ns]
 6   DATA_VENCIMENTO  39 non-null     datetime64[ns]
 7   NUMERO_CERTIDAO  39 non-null     object        
 8   NADA_CONSTA      39 non-null     bool          
 9   CREATED_AT       39 non-null     datetime64[ns]
 10  UPDATED_AT       39 non-null     datetime64[ns]
dtypes: bool(1), datetime64[ns](4), object(6)
memory usage: 3.2+ KB


,CERTIDAO_ID,CLIENTE_ID,PASTA_DESTINO,ARQUIVO,STATUS,DATA_INICIO,DATA_VENCIMENTO,NUMERO_CERTIDAO,NADA_CONSTA,CREATED_AT,UPDATED_AT
0,CEF-CRFGTS,AUTOLTDA,clientes/autodata/,2024-11-10_CR FGTS.pdf,VENCIDA,2024-09-30,2024-11-10,SZFHLST1NP,True,2024-11-16 02:51:45,2024-11-17 01:26:31.440307
1,TJDFT-CNFR,AUTOLTDA,clientes/autodata/,2024-10-09_CN Falencia e Recuperacao.pdf,VENCIDA,2024-09-09,2024-10-09,85J6FHJEPA,True,2024-11-16 02:51:45,2024-11-17 01:26:31.462602
2,CGU-CNC,AUTOLTDA,clientes/autodata/,"2024-11-12_CN Correcional (ePAD, CGU-PJ, CEIS,...",VENCIDA,2024-09-13,2024-11-12,XJTS5TBKHE,True,2024-11-16 02:51:45,2024-11-17 01:26:31.463605
3,FN-CNDTF,AUTOLTDA,clientes/autodata/,2024-11-13_CND Tributos Federais.pdf,VENCIDA,2024-09-06,2024-11-13,JLKPCGL1UU,True,2024-11-16 02:51:45,2024-11-17 01:26:31.463605
4,GDF-CND,AUTOLTDA,clientes/autodata/,2024-11-17_CND GDF.pdf,VENCIDA,2024-09-20,2024-11-17,LKTPKFYZNQ,True,2024-11-16 02:51:45,2024-11-17 01:26:31.463605


In [215]:
atualizar_planilha(gestao_monitoramento, workbook, df_monitor)

Arquivo Excel atualizado com os dados do DataFrame!


### vencida

In [216]:
df_monitor = pd.read_excel(gestao_monitoramento, sheet_name='db_monitor')
df_monitor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   CERTIDAO_ID      39 non-null     object        
 1   CLIENTE_ID       39 non-null     object        
 2   PASTA_DESTINO    39 non-null     object        
 3   ARQUIVO          39 non-null     object        
 4   STATUS           39 non-null     object        
 5   DATA_INICIO      39 non-null     datetime64[ns]
 6   DATA_VENCIMENTO  39 non-null     datetime64[ns]
 7   NUMERO_CERTIDAO  39 non-null     object        
 8   NADA_CONSTA      39 non-null     bool          
 9   CREATED_AT       39 non-null     datetime64[ns]
 10  UPDATED_AT       39 non-null     datetime64[ns]
dtypes: bool(1), datetime64[ns](4), object(6)
memory usage: 3.2+ KB


In [217]:
for index, row in df_monitor.iterrows():
    horario_atual = datetime.now()
    if row['DATA_VENCIMENTO'] <= horario_atual:
        df_monitor.at[index, 'STATUS'] = 'VENCIDA'
        df_monitor.at[index, 'UPDATED_AT'] = horario_atual

In [218]:
df_monitor

,CERTIDAO_ID,CLIENTE_ID,PASTA_DESTINO,ARQUIVO,STATUS,DATA_INICIO,DATA_VENCIMENTO,NUMERO_CERTIDAO,NADA_CONSTA,CREATED_AT,UPDATED_AT
0,CEF-CRFGTS,AUTOLTDA,clientes/autodata/,2024-11-10_CR FGTS.pdf,VENCIDA,2024-09-30,2024-11-10,SZFHLST1NP,True,2024-11-16 02:51:45,2024-11-17 01:26:31.440307
1,TJDFT-CNFR,AUTOLTDA,clientes/autodata/,2024-10-09_CN Falencia e Recuperacao.pdf,VENCIDA,2024-09-09,2024-10-09,85J6FHJEPA,True,2024-11-16 02:51:45,2024-11-17 01:26:31.462602
2,CGU-CNC,AUTOLTDA,clientes/autodata/,"2024-11-12_CN Correcional (ePAD, CGU-PJ, CEIS,...",VENCIDA,2024-09-13,2024-11-12,XJTS5TBKHE,True,2024-11-16 02:51:45,2024-11-17 01:26:31.463605
3,FN-CNDTF,AUTOLTDA,clientes/autodata/,2024-11-13_CND Tributos Federais.pdf,VENCIDA,2024-09-06,2024-11-13,JLKPCGL1UU,True,2024-11-16 02:51:45,2024-11-17 01:26:31.463605
4,GDF-CND,AUTOLTDA,clientes/autodata/,2024-11-17_CND GDF.pdf,VENCIDA,2024-09-20,2024-11-17,LKTPKFYZNQ,True,2024-11-16 02:51:45,2024-11-17 01:26:31.463605
5,GDF-DIF,AUTOLTDA,clientes/autodata/,2024-10-11_Comprovante DIF GDF.pdf,VENCIDA,2024-09-11,2024-10-11,-não existe-,False,2024-11-16 02:51:45,2024-11-17 01:26:31.464604
6,TST-CNDT,AUTOLTDA,clientes/autodata/,2024-11-14_CND Trabalhista.pdf,VENCIDA,2024-10-02,2024-11-14,QLHVUINIOF,True,2024-11-16 02:51:45,2024-11-17 01:26:31.464604
7,CNJ-CNIADM,AUTOLTDA,clientes/autodata/,2024-10-25_CN Impropidade Administrativa e Ine...,VENCIDA,2024-09-25,2024-10-25,672D.DF85.D1C3.5589,True,2024-11-16 02:51:45,2024-11-17 01:26:31.465174
8,CEF-CRFGTS,BIGBOX,clientes/bigbox/,2024-12-07_CR FGTS.pdf,monitorando,2024-11-08,2024-12-07,2024110818421197249619,True,2024-11-16 02:51:45,2024-11-17 01:22:21.911000
9,TJDFT-CNFR,BIGBOX,clientes/bigbox/,2024-12-14_CN Falencia e Recuperacao.pdf,monitorando,2024-11-14,2024-12-14,2024.CTD.F2H1.0WMJ.OFPU.E6S2.0YBG,True,2024-11-16 02:51:45,2024-11-17 01:22:21.911000
